## Selecting "riparian influenced VRI"

- If black cottonwood (ACT) is present, then the forest is classified as riparian influenced. 

- If the poplar (AC), and balsam poplar (ACB) percent cover sum to a value greater than any other species within a polygon, then the forest is classified as a riparian influenced stand.  

- If the paper birch (EP) percent cover is greater than any other species within a polygon, and there is either poplar, or balsam poplar, then the forest is classified as a riparian influenced stand.

In [28]:
import duckdb, os
%load_ext sql
workingdir = r'C:\Working\WMBAnalysis'
db = os.path.join(workingdir,'vri_analysis.db')

conn = duckdb.connect(database=db)
conn.load_extension("spatial")

%sql conn --alias duckdb

The sql extension is already loaded. To reload it, use:
  %reload_ext sql
There's a new jupysql version available (0.10.16), you're running 0.10.12. To upgrade: pip install jupysql --upgrade
Deploy Dash apps for free on Ploomber Cloud! Learn more: https://ploomber.io/s/signup


In [27]:
%%sql
# Riparian influenced forest in study area
select COUNT(OBJECTID),
    SUM(POLYGON_AREA)
from (
    SELECT OBJECTID, POLYGON_AREA,
    CONCAT(SPECIES_CD_1, ' ', SPECIES_CD_2, ' ', SPECIES_CD_3, ' ', SPECIES_CD_4, ' ', SPECIES_CD_5, ' ', SPECIES_CD_6, ' ') AS SpcList,
    (CASE WHEN SPECIES_CD_1 IN ('AC', 'ACB') THEN SPECIES_PCT_1 ELSE 0 END +
        CASE WHEN SPECIES_CD_2 IN ('AC', 'ACB') THEN SPECIES_PCT_2 ELSE 0 END +
        CASE WHEN SPECIES_CD_3 IN ('AC', 'ACB') THEN SPECIES_PCT_3 ELSE 0 END +
        CASE WHEN SPECIES_CD_4 IN ('AC', 'ACB') THEN SPECIES_PCT_4 ELSE 0 END +
        CASE WHEN SPECIES_CD_5 IN ('AC', 'ACB') THEN SPECIES_PCT_5 ELSE 0 END +
        CASE WHEN SPECIES_CD_6 IN ('AC', 'ACB') THEN SPECIES_PCT_6 ELSE 0 END ) AS poplarPct
    FROM VRI
WHERE 
        SpcList LIKE '%ACT%'
        
    OR poplarPct >= SPECIES_PCT_1  # Poplar percent more than the greatest species. 
    
    OR SPECIES_CD_1 = 'EP' AND SpcList LIKE '%AC %'
    OR SPECIES_CD_1 = 'EP' AND SpcList LIKE '%ACB%'
)

Running query in 'duckdb'

count(OBJECTID),sum(POLYGON_AREA)
3172,32485.53169836179


In [38]:
df = conn.sql("""SELECT OBJECTID, ST_AsText(GEOM) AS wkt, POLYGON_AREA,
    CONCAT(SPECIES_CD_1, ' ', SPECIES_CD_2, ' ', SPECIES_CD_3, ' ', SPECIES_CD_4, ' ', SPECIES_CD_5, ' ', SPECIES_CD_6, ' ') AS SpcList,
    (CASE WHEN SPECIES_CD_1 IN ('AC', 'ACB') THEN SPECIES_PCT_1 ELSE 0 END +
        CASE WHEN SPECIES_CD_2 IN ('AC', 'ACB') THEN SPECIES_PCT_2 ELSE 0 END +
        CASE WHEN SPECIES_CD_3 IN ('AC', 'ACB') THEN SPECIES_PCT_3 ELSE 0 END +
        CASE WHEN SPECIES_CD_4 IN ('AC', 'ACB') THEN SPECIES_PCT_4 ELSE 0 END +
        CASE WHEN SPECIES_CD_5 IN ('AC', 'ACB') THEN SPECIES_PCT_5 ELSE 0 END +
        CASE WHEN SPECIES_CD_6 IN ('AC', 'ACB') THEN SPECIES_PCT_6 ELSE 0 END ) AS poplarPct
    FROM VRI
WHERE 
        SpcList LIKE '%ACT%'
        
    OR poplarPct >= SPECIES_PCT_1  
    
    OR SPECIES_CD_1 = 'EP' AND SpcList LIKE '%AC %'
    OR SPECIES_CD_1 = 'EP' AND SpcList LIKE '%ACB%';""").to_df()

In [39]:
import geopandas as gpd
df['geometry'] = gpd.GeoSeries.from_wkt(df['wkt'])
gdf = gpd.GeoDataFrame(df.drop(columns='wkt'), geometry=df['geometry'])
gdf

,OBJECTID,POLYGON_AREA,SpcList,poplarPct,geometry
0,90850,2.096588,AC SX,90.0,"MULTIPOLYGON (((1249854.390 1284674.310, 12498..."
1,91440,14.860696,AC SX AT PL,55.0,"MULTIPOLYGON (((1214802.140 1301698.300, 12147..."
2,82685,1.757958,SW AT ACT,0.0,"MULTIPOLYGON (((1225412.370 1296850.890, 12253..."
3,83282,30.076598,SW EP ACT,0.0,"MULTIPOLYGON (((1279996.210 1454561.930, 12799..."
4,85422,8.279051,ACT SW,0.0,"MULTIPOLYGON (((1265368.660 1374283.390, 12653..."
...,...,...,...,...,...
3167,5722599,23.204101,AT SW ACT,0.0,"MULTIPOLYGON (((1245432.984 1293642.774, 12454..."
3168,5756074,6.261917,SW AT ACT,0.0,"MULTIPOLYGON (((1272420.840 1453222.440, 12724..."
3169,5756372,4.762884,ACT SW,0.0,"MULTIPOLYGON (((1244446.410 1310495.940, 12444..."
3170,5757001,15.461019,SW ACT,0.0,"MULTIPOLYGON (((1262257.620 1463541.550, 12622..."


In [41]:
gdf.to_file(r'\\spatialfiles.bcgov\work\srm\fsj\Workarea\nross\WMBPlanning\Data\Riparian\RiparianForest.shp')

C:\Users\nross\AppData\Local\Temp\ipykernel_15088\54527671.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf.to_file(r'\\spatialfiles.bcgov\work\srm\fsj\Workarea\nross\WMBPlanning\Data\Riparian\RiparianForest.shp')


In [42]:
conn.close()

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv(r'\\spatialfiles.bcgov\work\srm\fsj\Workarea\nross\WMBPlanning\Data\habwmb.csv')

In [23]:
df_1000

,Unnamed: 0,AreaHa,WMBName,WMBAreaHa,cat,PctWMB
122,123,191.699829,Blueberry River,296019.443572,Broad,0.000648
291,292,791.801740,Blueberry River,296019.443572,Broad,0.002675
364,365,137.989189,Blueberry River,296019.443572,Broad,0.000466
439,440,299.208462,Blueberry River,296019.443572,Broad,0.001011
444,445,355.585350,Blueberry River,296019.443572,Broad,0.001201
...,...,...,...,...,...,...
9264,9265,2407.717043,Lower Sikanni Chief River,309789.456792,Broad,0.007772
9268,9269,120.527844,Lower Sikanni Chief River,309789.456792,Broad,0.000389
9271,9272,5893.934333,Lower Sikanni Chief River,309789.456792,Broad,0.019026
9272,9273,30777.800427,Lower Sikanni Chief River,309789.456792,Broad,0.099351


In [26]:
df_1000 = df.loc[(df['AreaHa'] > 500) & (df['cat'] == 'Refined')]

df_1000[['WMBName', 'AreaHa']].groupby('WMBName').count().join(
    df_1000[['WMBName', 'AreaHa', 'PctWMB']].groupby('WMBName').sum(), rsuffix='_'
    )


,AreaHa,AreaHa_,PctWMB
WMBName,,,
Blueberry River,4,4186.348596,0.014142
Lower Sikanni Chief River,31,57379.260172,0.185220
Middle Beatton River,8,7122.658115,0.079332
Upper Beatton River,11,13607.478652,0.040063


In [28]:
df = pd.read_csv(r"\\spatialfiles.bcgov\work\srm\fsj\Workarea\nross\WMBPlanning\Data\Riparian\Riparian_Max_byWMB.csv")
df.rename(columns={"WATER_MANAGEMENT_BASIN_NAME": "WMBName", "Hectares": "WMBAreaHa"}, inplace=True)
df["AreaHa"] = df["Shape_Area"]/10000
df["Pct WMB"] = (df['AreaHa']/df['WMBAreaHa'])
df.columns

Index(['FID_Riparian_Union_Max_Dissolve_',
       'FWA_Lines_NoPolys_Dissolve_STREAM_ORDER', 'Max_Source', 'AreaHa',
       'FID_Priority_WMB', 'OG_WATER_MANAGEMENT_BASIN_ID', 'WMBName',
       'Claim_Area', 'WMBAreaHa', 'Shape_Length', 'Shape_Area', 'Pct WMB'],
      dtype='object')

In [29]:
grp = df[["Max_Source", "WMBName", "AreaHa", "Pct WMB"]].groupby(["WMBName", "Max_Source"]).sum()

grp.sort_values(by=["WMBName", 'AreaHa'], axis = 0, ascending=False)

AreaHa   Pct WMB
WMBName                   Max_Source                                  
Upper Beatton River       Stream Buffer          6486.674638  0.019098
                          River Buffer           5141.843885  0.015139
                          Stream Buffer (3-6)    5098.835098  0.015012
                          Wetland Stream Buffer  1258.929021  0.003707
                          Lake Buffer            1027.184547  0.003024
                          Riparian Forest         508.620541  0.001497
Middle Beatton River      River Buffer           1981.647820  0.022071
                          Stream Buffer          1292.676135  0.014398
                          Stream Buffer (3-6)    1109.417948  0.012357
                          Wetland Stream Buffer   510.397992  0.005685
                          Lake Buffer             490.133871  0.005459
                          Riparian Forest         179.291405  0.001997
Lower Sikanni Chief River Stream Buffer          5294.608777  0.017091
                          Stream Buffer (3-6)    4398.008265  0.014197
                          River Buffer           4046.518784  0.013062
                          Riparian Forest        3256.086780  0.010511
                          Wetland Stream Buffer  2108.096031  0.006805
                          Lake Buffer            2017.244254  0.006512
Blueberry River           Stream Buffer (3-6)    5268.514421  0.017798
                          Stream Buffer          5051.772405  0.017066
                          Riparian Forest        2811.817054  0.009499
                          River Buffer           2141.361153  0.007234
                          Wetland Stream Buffer  1046.505948  0.003535
                          Lake Buffer             579.535691  0.001958